In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras import callbacks
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [2]:
val_metrics = np.load("val_metric_dict_1.npy", allow_pickle=True)

In [3]:
indexes=[]
for j in range(1, 6):
    for i in range(1,11):
        indexes.append('month '+str(j)+' lookahead ' + str(i))

In [4]:
val_df = pd.DataFrame(val_metrics.reshape(val_metrics.shape[0], val_metrics.shape[2]), columns = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2'])

In [5]:
val_df.index=indexes[:len(val_df)]

In [6]:
val_df

,RMSE,MSE,MAE,MAPE,R2
month 1 lookahead 1,12.376245,153.171430,7.559685,0.108956,0.732118
month 1 lookahead 2,12.551234,157.533487,7.567140,0.108327,0.724490
month 1 lookahead 3,12.692354,161.095839,7.857314,0.112133,0.718379
month 1 lookahead 4,12.536113,157.154126,7.835077,0.114042,0.725270
month 1 lookahead 5,12.681144,160.811410,8.049060,0.118607,0.718877
month 1 lookahead 6,12.590872,158.530053,7.959245,0.117496,0.722865
month 1 lookahead 7,12.785877,163.478641,8.193432,0.121809,0.714347
month 1 lookahead 8,12.778948,163.301507,8.086656,0.117917,0.714657
month 1 lookahead 9,12.692485,161.099179,7.935059,0.114795,0.718505
month 1 lookahead 10,12.919996,166.926287,8.199559,0.118975,0.708444


In [80]:
val_df.groupby(np.arange(len(val_df))//10).mean()

,RMSE,MSE,MAE,MAPE,R2
0,12.660527,160.310196,7.924223,0.115306,0.719795
1,36.629189,1343.708835,18.908185,0.122014,0.678165
2,35.427725,1255.490425,12.757488,0.159714,0.486067
3,9.501184,90.298839,6.141839,0.154470,0.730133
4,16.106481,260.733233,8.210320,0.160792,0.769611


In [91]:
averaged_months= val_df.iloc[0::10,:].mean().to_frame().transpose()
for i in range(1,10):
    averaged_months = averaged_months.append((val_df.iloc[i::10,:]).mean(), ignore_index=True)

In [92]:
averaged_months

,RMSE,MSE,MAE,MAPE,R2
0,21.293357,579.563224,10.140989,0.129148,0.698510
1,21.370282,579.450893,10.352163,0.135814,0.692253
2,21.457688,583.541387,10.351700,0.133694,0.690397
3,21.628675,596.448326,10.591564,0.139473,0.685656
4,22.089068,625.694309,10.994138,0.148265,0.676206
5,22.002451,622.056791,10.799970,0.144310,0.678955
6,22.256522,630.631702,10.981037,0.149471,0.671754
7,23.207989,683.265728,11.424316,0.152276,0.647399
8,22.680103,663.033506,11.162503,0.146828,0.663497
9,22.664077,657.397190,11.085727,0.145311,0.662915


RMSE     21.293357
MSE     579.563224
MAE      10.140989
MAPE      0.129148
R2        0.698510
RMSE     21.370282
MSE     579.450893
MAE      10.352163
MAPE      0.135814
R2        0.692253
dtype: float64